## Download pretrained model XLNET-Bahasa

You can get it from here, https://github.com/huseinzol05/Malaya/tree/master/pretrained-model/xlnet#download

**In this example, I am going to use [SMALL XLNET](https://huseinhouse-storage.s3-ap-southeast-1.amazonaws.com/bert-bahasa/xlnet-bahasa-small.tar.gz)**

In [23]:
# uncomment all to download and install dependencies

# !wget https://huseinhouse-storage.s3-ap-southeast-1.amazonaws.com/bert-bahasa/xlnet-bahasa-small.tar.gz
# !tar -zxf xlnet-bahasa-small.tar.gz
# !wget https://raw.githubusercontent.com/huseinzol05/NLP-Models-Tensorflow/master/topic-model/modeling.py
# !wget https://raw.githubusercontent.com/huseinzol05/NLP-Models-Tensorflow/master/topic-model/prepro_utils.py
# !wget https://raw.githubusercontent.com/huseinzol05/NLP-Models-Tensorflow/master/topic-model/xlnet.py
# !wget https://raw.githubusercontent.com/zihangdai/xlnet/master/model_utils.py
# !pip3 install sentencepiece

## Download dataset

In this example, I am going to use small [subjectivity dataset](https://github.com/huseinzol05/Malaya-Dataset/tree/master/subjectivity), only got 2 classes `negative` and `positive`.

**Please use your own dataset**.

In [7]:
# !wget https://raw.githubusercontent.com/huseinzol05/Malaya-Dataset/master/subjectivity/subjectivity-negative-bm.txt

In [8]:
with open('subjectivity-negative-bm.txt') as fopen:
    texts = fopen.read().split('\n')[:-1]
len(texts)

4989

In [9]:
texts[0]

'filem bermula pada masa lalu di mana seorang budak lelaki bernama sam cuba menyelamatkan celebi dari pemburu'

In [3]:
!ls

modeling.py	    xlnet-bahasa-small.tar.gz
prepro_utils.py     xlnet.py
xlnet-bahasa-small  xlnet-transfer-learning-classification.ipynb


In [24]:
import xlnet
import numpy as np
import tensorflow as tf
import model_utils
from tqdm import tqdm

In [13]:
from prepro_utils import preprocess_text, encode_ids
import sentencepiece as spm

def tokenize_fn(text):
    text = preprocess_text(text, lower= False)
    return encode_ids(sp_model, text)

sp_model = spm.SentencePieceProcessor()
sp_model.Load('xlnet-bahasa-small/sp10m.cased.v5.model')

True

In [10]:
# can increase as much as you want, again, attention size will increase!
MAX_SEQ_LENGTH = 128

SEG_ID_A   = 0
SEG_ID_B   = 1
SEG_ID_CLS = 2
SEG_ID_SEP = 3
SEG_ID_PAD = 4

special_symbols = {
    "<unk>"  : 0,
    "<s>"    : 1,
    "</s>"   : 2,
    "<cls>"  : 3,
    "<sep>"  : 4,
    "<pad>"  : 5,
    "<mask>" : 6,
    "<eod>"  : 7,
    "<eop>"  : 8,
}

VOCAB_SIZE = 32000
UNK_ID = special_symbols["<unk>"]
CLS_ID = special_symbols["<cls>"]
SEP_ID = special_symbols["<sep>"]
MASK_ID = special_symbols["<mask>"]
EOD_ID = special_symbols["<eod>"]

In [14]:
input_ids, input_masks, segment_ids = [], [], []
for text in tqdm(texts):
    tokens_a = tokenize_fn(text)
    if len(tokens_a) > MAX_SEQ_LENGTH - 2:
        tokens_a = tokens_a[:(MAX_SEQ_LENGTH - 2)]
        
    tokens = []
    segment_id = []
    for token in tokens_a:
        tokens.append(token)
        segment_id.append(SEG_ID_A)
    tokens.append(SEP_ID)
    segment_id.append(SEG_ID_A)
    tokens.append(CLS_ID)
    segment_id.append(SEG_ID_CLS)
    
    input_id = tokens
    input_mask = [0] * len(input_id)
    if len(input_id) < MAX_SEQ_LENGTH:
        delta_len = MAX_SEQ_LENGTH - len(input_id)
        input_id = [0] * delta_len + input_id
        input_mask = [1] * delta_len + input_mask
        segment_id = [SEG_ID_PAD] * delta_len + segment_id
    
    input_ids.append(input_id)
    input_masks.append(input_mask)
    segment_ids.append(segment_id)

100%|██████████| 4989/4989 [00:00<00:00, 6237.21it/s]


In [17]:
kwargs = dict(
      is_training=True,
      use_tpu=False,
      use_bfloat16=False,
      dropout=0.0,
      dropatt=0.0,
      init='normal',
      init_range=0.1,
      init_std=0.05,
      clamp_len=-1)

xlnet_parameters = xlnet.RunConfig(**kwargs)
xlnet_config = xlnet.XLNetConfig(json_path='xlnet-bahasa-small/config.json')

In [18]:
epoch = 10
batch_size = 60
warmup_proportion = 0.1
num_train_steps = int(len(input_ids) / batch_size * epoch)
num_warmup_steps = int(num_train_steps * warmup_proportion)
print(num_train_steps, num_warmup_steps)
learning_rate = 2e-5

training_parameters = dict(
      decay_method = 'poly',
      train_steps = num_train_steps,
      learning_rate = learning_rate,
      warmup_steps = num_warmup_steps,
      min_lr_ratio = 0.0,
      weight_decay = 0.00,
      adam_epsilon = 1e-8,
      num_core_per_host = 1,
      lr_layer_decay_rate = 1,
      use_tpu=False,
      use_bfloat16=False,
      dropout=0.0,
      dropatt=0.0,
      init='normal',
      init_range=0.1,
      init_std=0.05,
      clip = 1.0,
      clamp_len=-1,)

831 83


In [19]:
class Parameter:
    def __init__(self, decay_method, warmup_steps, weight_decay, adam_epsilon, 
                num_core_per_host, lr_layer_decay_rate, use_tpu, learning_rate, train_steps,
                min_lr_ratio, clip, **kwargs):
        self.decay_method = decay_method
        self.warmup_steps = warmup_steps
        self.weight_decay = weight_decay
        self.adam_epsilon = adam_epsilon
        self.num_core_per_host = num_core_per_host
        self.lr_layer_decay_rate = lr_layer_decay_rate
        self.use_tpu = use_tpu
        self.learning_rate = learning_rate
        self.train_steps = train_steps
        self.min_lr_ratio = min_lr_ratio
        self.clip = clip
        
training_parameters = Parameter(**training_parameters)

In [25]:
class Model:
    def __init__(
        self,
        dimension_output,
        learning_rate = 2e-5,
    ):
        self.X = tf.placeholder(tf.int32, [None, None])
        self.segment_ids = tf.placeholder(tf.int32, [None, None])
        self.input_masks = tf.placeholder(tf.float32, [None, None])
        self.Y = tf.placeholder(tf.int32, [None])
        
        xlnet_model = xlnet.XLNetModel(
            xlnet_config=xlnet_config,
            run_config=xlnet_parameters,
            input_ids=tf.transpose(self.X, [1, 0]),
            seg_ids=tf.transpose(self.segment_ids, [1, 0]),
            input_mask=tf.transpose(self.input_masks, [1, 0]))
        
        summary = xlnet_model.get_pooled_out("last", True)
        print(summary)
        
        self.logits = tf.layers.dense(summary, dimension_output)
        
        self.cost = tf.reduce_mean(
            tf.nn.sparse_softmax_cross_entropy_with_logits(
                logits = self.logits, labels = self.Y
            )
        )
        self.optimizer, self.learning_rate, _ = model_utils.get_train_op(training_parameters, self.cost)
        
        correct_pred = tf.equal(
            tf.argmax(self.logits, 1, output_type = tf.int32), self.Y
        )
        self.accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [26]:
dimension_output = 2

tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Model(dimension_output, learning_rate)

sess.run(tf.global_variables_initializer())

INFO:tensorflow:memory input None
INFO:tensorflow:Use float type <dtype: 'float32'>


/home/husein/.local/lib/python3.6/site-packages/tensorflow/python/client/session.py:1735: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


Tensor("model_1/sequnece_summary/summary/Tanh:0", shape=(?, 256), dtype=float32)


Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



In [27]:
import collections
import re

def get_assignment_map_from_checkpoint(tvars, init_checkpoint):
    """Compute the union of the current variables and checkpoint variables."""
    assignment_map = {}
    initialized_variable_names = {}

    name_to_variable = collections.OrderedDict()
    for var in tvars:
        name = var.name
        m = re.match('^(.*):\\d+$', name)
        if m is not None:
            name = m.group(1)
        name_to_variable[name] = var

    init_vars = tf.train.list_variables(init_checkpoint)

    assignment_map = collections.OrderedDict()
    for x in init_vars:
        (name, var) = (x[0], x[1])
        if name not in name_to_variable:
            continue
        assignment_map[name] = name_to_variable[name]
        initialized_variable_names[name] = 1
        initialized_variable_names[name + ':0'] = 1

    return (assignment_map, initialized_variable_names)

In [28]:
tvars = tf.trainable_variables()
checkpoint = 'xlnet-bahasa-small/model.ckpt'
assignment_map, initialized_variable_names = get_assignment_map_from_checkpoint(tvars, 
                                                                                checkpoint)

In [29]:
saver = tf.train.Saver(var_list = assignment_map)
saver.restore(sess, checkpoint)

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from xlnet-bahasa-small/model.ckpt


## Below should be your training session

Tensorflow training session as usual, good luck!